In [54]:
from google.colab import files
uploaded = files.upload()

Saving bread.xlsx - Sheet1.csv to bread.xlsx - Sheet1 (2).csv


In [55]:
import numpy as np
import pandas as pd
from itertools import combinations
from mlxtend.preprocessing import TransactionEncoder


In [56]:
df = pd.read_csv("bread.xlsx - Sheet1.csv")
df

,"Date,Time,Transaction,Item"
0,"2016-10-30,09:58:11,1,Bread"
1,"2016-10-30,10:05:34,2,Scandinavian"
2,"2016-10-30,10:05:34,2,Scandinavian"
3,"2016-10-30,10:07:57,3,Hot chocolate"
4,"2016-10-30,10:07:57,3,Jam"
...,...
21288,"2017-04-09,14:32:58,9682,Coffee"
21289,"2017-04-09,14:32:58,9682,Tea"
21290,"2017-04-09,14:57:06,9683,Coffee"
21291,"2017-04-09,14:57:06,9683,Pastry"


In [57]:
#dropping the duplicate values
df=df.drop_duplicates()

In [58]:
#splitting the values into tables
df2=df['Date,Time,Transaction,Item'].str.split(',', n = 3, expand = True)

In [59]:
#assigning columns name
import warnings
warnings.filterwarnings('ignore')

df['Date'] = df2[0]
df['Time'] = df2[1]
df['Transaction'] = df2[2]
df['Item'] = df2[3]

In [60]:
#the main dataframe we need
df[['Transaction', 'Item']]

,Transaction,Item
0,1,Bread
1,2,Scandinavian
3,3,Hot chocolate
4,3,Jam
5,3,Cookies
...,...,...
21288,9682,Coffee
21289,9682,Tea
21290,9683,Coffee
21291,9683,Pastry


In [61]:
#converting the table into a matrix form
df3 = pd.crosstab(index= df['Transaction'], columns= df['Item'])
df3

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
#dropping the column
df3 = df3.drop(['NONE'], axis = 1)

In [63]:
#making the apriori function
def apriori_created(dataset, min_support,  max_length):
    import numpy as np
    import pandas as pd
    from itertools import combinations

    support = {} 
    List = list(dataset.columns)

    for i in range(1, max_length+1):
        c = set(combinations(List,i))
        
        List =set()     

        for j in list(c):
            sup = dataset.loc[:,j].product(axis=1).sum()/len(dataset.index)
            if sup > min_support:
                support[j] = sup
 
                List = list(set(List) | set(j))

    result = pd.DataFrame(list(support.items()), columns = ["Items", "Support"])
    return(result)

In [64]:
#making the associative rule function
def associative_rule_created(dataset, min_threshold):
    import pandas as pd
    from itertools import permutations
 
    support = pd.Series(dataset.Support.values, index=dataset.Items).to_dict()
    data = []
    L= dataset.Items.values

    p = list(permutations(L, 2))

    for i in p:

        if set(i[0]).issubset(i[1]):
            conf = support[i[1]]/support[i[0]]
            if conf > min_threshold:
                j = i[1][not i[1].index(i[0][0])]
                lift = support[i[1]]/(support[i[0]]* support[(j,)])
                leverage = support[i[1]] - (support[i[0]]* support[(j,)])
                convection = (1 - support[(j,)])/(1- conf)
                data.append([i[0], (j,), support[i[0]], support[(j,)], support[i[1]], conf, lift, leverage, convection])

    result = pd.DataFrame(data, columns = ["antecedents", "consequents", "antecedent support", "consequent support",
                                        "support", "confidence", "Lift", "Leverage", "Convection"])
    return(result)

In [65]:
#finding the min support values for our dataset
df4 = apriori_created(df3, 0.04, 3)
df4.sort_values(by = 'Support', ascending = False)

,Items,Support
0,"(Coffee,)",0.475081
1,"(Bread,)",0.324940
6,"(Tea,)",0.141643
3,"(Cake,)",0.103137
12,"(Coffee, Bread)",0.089393
4,"(Pastry,)",0.085510
5,"(Sandwich,)",0.071346
8,"(Medialuna,)",0.061379
7,"(Hot chocolate,)",0.057916
9,"(Coffee, Cake)",0.054349


In [66]:
#finding the min confidence values for our dataset
df5 = associative_rule_created(df4, 0.5)
df5

,antecedents,consequents,antecedent support,consequent support,support,confidence,Lift,Leverage,Convection
0,"(Cake,)","(Coffee,)",0.103137,0.475081,0.054349,0.526958,1.109196,0.00535,1.109667
1,"(Pastry,)","(Coffee,)",0.085510,0.475081,0.047214,0.552147,1.162216,0.00659,1.172079
